<a href="https://colab.research.google.com/github/mhtn-it/6_APP_Project/blob/main/Report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Đồ án môn Lập trình song song ứng dụng
**Giảng viên hướng dẫn: Thầy Trần Trung Kiên**

## Thông tin nhóm
### STT: Nhóm 6

### Thành viên:
1. 1712258 - Nguyễn Văn Hậu - [kenneth-nguyenn](https://github.com/kenneth-nguyenn)
2. 18120181 - Nguyễn Thị Cẩm Hồng - [chnhgr](https://github.com/chnhgr)
3. 18120216 - Mai Huỳnh Trung Nguyên - [mhtn-it](https://github.com/mhtn-it)

### Tài liệu của nhóm
- Link thùng chứa Github của nhóm: [Github](https://github.com/mhtn-it/6_APP_Project)
- Kế hoạch nhóm, phân chia công việc: [Google Sheet](https://docs.google.com/spreadsheets/d/1lNRWbRRnsN0L1bEBLm2tHkauJC2jS9DPZiZfhLQ9Av8/edit?usp=sharing)
- Link Colab thực thi: [Google Colab](https://colab.research.google.com/github/mhtn-it/6_APP_Project/blob/main/Report.ipynb)

## **1. Mô tả ứng dụng:** 

### **1.1. Mô tả tổng quan:**

**Đề tài**: Thay đổi background ảnh dựa vào phương pháp Poisson Matting

**Input**: 
- Một bức ảnh chân dung
- Trimap của ảnh (được tạo từ một bài toán khác)
- Ảnh nền mới mong muốn

**Output**: 
- Ảnh đã được thay đổi nền

**Ý nghĩa thực tế của ứng dụng**:
- Thay đổi background của ảnh khi cần thiết, ví dụ trong các trường hợp như ảnh kỷ niệm, ảnh thẻ, trang trí...
- Áp dụng trên các phần mềm chỉnh sửa ảnh, trang web chỉnh sửa ảnh online
- Là một phần xử lý của quy trình đổi nền của video

**Lý do cần tăng tốc**: 
- Toàn bộ quá trình xử lý để cho ra một bức ảnh đúng và chính xác có thể mất đến vài phút (giảm trải nghiệm người dùng)
- Khi xử lý hàng loạt, số lượng lớn thì thời gian sẽ rất lâu
- Nếu áp dụng cho đổi nền video thì cần có tốc độ nhanh

### **1.2. Ý tưởng chính của bài toán:**

Từ hình ảnh gốc, trimap, ảnh nền mới ứng dụng sử dụng các biện pháp phủ (matting) và pha trộn (blending) để ghép ảnh chân dung background mới.

<img src='https://drive.google.com/uc?id=1rpQhdF5U7Y2LTdbPb9pLc-i6dJ2IS3zC'/>

Qua nhiều phương pháp cải thiện thì foreground ngày càng chính xác với đối tượng của mình.

<img src='https://drive.google.com/uc?id=1sdEYodB2lyyb_FROl5okKbeavDmTJJLr'/>

*Nguồn ảnh: từ Github [1] nhóm tham khảo*

### **1.3. Phần nhóm dự định song song hoá:**

Nhóm nhận thấy việc xử lý ảnh có thể song song hoá bằng cách chia nhỏ công việc cho từng pixel. Do đó nhóm dự định sẽ lựa chọn bước: Thao tác Global Matting dùng để tính giá trị alpha cho việc blending ảnh.

## **2. Phần bổ trợ**

Phần này dùng để tạo cách hàm bổ trợ cho việc thực hiện trên Google Colab. Bao gồm: liên kết Google Colab với Drive, kiểm tra thông tin GPU, chuyển thư mục làm việc về thư mục `Code`, Import thư viện.

Kiểm tra thông tin GPU

In [1]:
from numba import jit, prange, cuda
cuda.detect()

Found 1 CUDA devices
id 0             b'Tesla T4'                              [SUPPORTED]
                      compute capability: 7.5
                           pci device id: 4
                              pci bus id: 0
Summary:
	1/1 devices are supported


True

Liên kết colab với Google Drive để tiết kiệm thời gian load dữ liệu.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Chuyển vị trí thực thi về thư mục Code

In [3]:
cd drive/MyDrive/6_APP_Project/Code

/content/drive/.shortcut-targets-by-id/1bDjdUTDdbr1EV_9VxSVIOGB60OutJlZi/6_APP_Project/Code


Import thư viện

In [4]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
import os
from  scipy import ndimage
import time
import scipy
from scipy.optimize import fmin
IMAGES_FOLDER_PATH = "../Data/Input"

## **3. Cài đặt tuần tự**

### **3.1. Thiết kế:**

#### 3.1.1. Quy trình tổng quan:

<img src='https://drive.google.com/uc?id=1PF-AdVz4KhFx2CmqpLdORmRBVEBED-xM'/>

<img src='https://drive.google.com/uc?id=1hnm-vuha0rYrlHrgK_rvJpSoMKYrKvZm'/>

<img src='https://drive.google.com/uc?id=1-FVmGkJIPUqEdUIRy12vYyyZP7MZVkM_'/>

<img src='https://drive.google.com/uc?id=1qY_tqmHLIExD4IJWn3uVjaCKoRSOdRJW'/>

#### 3.1.2. Các ảnh Input:

Danh sách các input, bao gồm:
- Ảnh gốc
- Ảnh trimap
- Ảnh background mới

*Các ảnh có kích thước là 480x640 pixels*

Lưu ý:
- Ảnh grayscale được thêm vào để minh hoạ, người dùng không cần cung cấp.
- Phương pháp tạo ảnh trimap nhóm đã tìm hiểu, làm thử nhưng chưa thành công và mất nhiều thời gian của nhóm. Do đó nhóm sử dụng các trimap có sẵn. Đưa việc tạo ra trimap sang một bài toán khác.

<img src='https://drive.google.com/uc?id=1ULspQFfkDyjg-WFZZnLmNRQCjKW1hhhJ'/>

#### 3.1.3. Phân tích ảnh input:

Từ các ảnh input ta dùng hàm tính toán mask để phân tích ra được vùng/ảnh của foreground, background và unknown. Nguyên tắc: các pixel trên ảnh trimap sẽ là:
- Background: nếu giá trị < 10.
- Foreground: nếu giá trị > 245.
- Unknown: còn lại

<img align="justify" src='https://drive.google.com/uc?id=1S36DiTkgMNj_aUYCNK8es4yvYu5dFmdA'/>


Từ các vùng ở trên ta ghép:
- F = foreground + unknown
- B = background + unknown

<img align="justify" src='https://drive.google.com/uc?id=1_uwZGhYmlF76OxKPetgzvz8ed24asQEJ'/>


#### 3.1.4. Global Matting:

**Phương trình Poisson:**

$$\nabla I = (F - B)\nabla \alpha + \alpha \nabla F + (1 - \alpha)\nabla B$$

Suy ra:

$$\Delta \alpha = div (\frac{\nabla I}{F-B})$$
trong đó, $\Delta = (\frac{\partial^2}{\partial x^2},\frac{\partial^2}{\partial y^2})$
là một toán tử Laplacian and toán tử vector div của Gradient ảnh input đối với hiệu của Foreground và Background.


Phương trinh Poisson được giải bằng phương pháp ước lượng Gauss Siedel Iteration

cho,  $$ \frac{\partial^2 f}{\partial x^2}+\frac{\partial^2 f}{\partial y^2} = S$$
$$ {f_{i,j}}^{(n+1)} = \frac{\beta}{4}({f_{i+1,j}}^{(n)} + {f_{i-1,j}}^{(n+1)} + {f_{i,j+1}}^{(n)} + {f_{i,j-1}}^{(n+1)} - S_{i,j}) + (1-\beta){f_{i,j}}^{(n)}$$
với $\beta$ nằm trong đoạn (1,2)

Nhóm hiện thực công thức trên bằng các hàm python để tính toán.

Tìm hiểu thêm về phương trình Poisson [tại đây](https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/poissonmatting_siggraph04.pdf)..


**Lý do sử dụng Phương trình Poisson:**

Poisson là một phương pháp tổng quát dùng để mô tả hành vi của các đại lượng ví dụ như hướng thế năng điện, trọng lực, chất lỏng. Áp dụng với bài toán này ta có thể mô tả xu hướng gradient của các pixel. Khi đó việc giải phương trình (tìm alpha) có thể xác định được độ mờ của pixel đó để biết nó có xu hướng nghiêng về background hay foreground.

Sau khi áp dụng phương pháp trên ta được một mảng numpy chứa các giá trị alpha dùng để ghép với nền mới. Ta có thể xem alpha nhưng một hệ số làm mờ. Ở công thức phía trên `f` chính là hệ số `alpha` mà ta cần tìm. 

<img src='https://drive.google.com/uc?id=1up3_IVpNZRzD-mziEwlr2M-Kjn_Xrh-4'/>


#### 3.1.5. Alpha Blending:

**Công thức tổng quan Bleding:**

Với mỗi pixel:

**I = α F + (1 − α )B**

Trong đó:
- I: là output
- α: alpha đã tính ở trên
- F: foreground
- B: background

Tiến hành blending mảng alpha global matting ở trên với hình nên mới để được kết quả.

<img src='https://drive.google.com/uc?id=1O9bYFhq6t6jfgrDaqglRTurH8NZt8xSm'/>

#### 3.1.6. Local Matting:

Người dùng sẽ quan sát ảnh sau khi dùng Global Matting và đánh dấu những điểm ROI (Region of Interest - Những khu vực mong muốn được cải thiện hơn) sau đó chương trình sẽ chạy Local Matting trên khu vực này để cải thiện chất lượng hình ảnh.
Local Matting cũng chính là Global Matting nhưng trên một khu vực nhỏ hơn.

Ví dụ, với hình ở trên ta chọn 4 khu vực có chỉ số như sau:
- top, bottom, left, right = [0, 20, 370, 435]
- top, bottom, left, right = [135, 155, 145, 170]
- top, bottom, left, right = [212, 240, 118, 150]
- top, bottom, left, right = [465, 480, 120, 150]

<img src='https://drive.google.com/uc?id=1s3DTgKOwGPqLOCmo_oZV0XncFOdW5Xfz'/>

Local matte và ảnh kết quả sau khi blending:

<img src="https://drive.google.com/uc?id=1St6RIQp6ztso5_k6gOSkqhCEmT_narm_"/>



Hiện tại nhóm đang tự quan sát và set cứng các giá trị ROI trong file code để dễ dàng và nhanh chóng kiểm thử. Nếu dự án ứng dụng vào thực tế thì nhóm đề xuất quy trình để xử lý:
- In ra ảnh từ global matting
- Người dùng nhập vào danh sách những khu vực muốn chỉnh sửa
- In ra kết quả sau khi local matting

Nhóm đã có làm thử nhưng vì mất nhiều thời gian nhập toạ độ nên lược bỏ trong report.

### **3.2. Đánh giá:**

#### **3.2.1. Kịch bản thí nghiệm:**

Chương trình sẽ lần lượt thực hiện theo quy trình sau:
1. Đọc ảnh gốc, ảnh tri map, ảnh nền mới. Chuyển ảnh gốc sang dạng grayscale
2. Chạy hàm `matting_combined`: hàm tổng hợp quy trình xử lý ảnh với Global Matting bên trên.
3. Lưu file ảnh output được áp dụng Global Matting.
4. Bắt đầu pha Local Matting. Cho người dùng nhập (ở đây nhóm set cứng) giá trị của vùng mong muốn được cải thiện.
5. Tính toán giá trị `alpha` mới của các vùng này.
6. Gọi hàm `alpha_blend` để áp giá trị `alpha` mới vào ảnh.
7. Lưu file ảnh output được áp dụng cả Global và Local Matting.



#### **3.2.2. Thực thi với Python (không jit):**

Hiện tại để tiện kiểm tra thì nhóm set cứng giá trị input (ảnh gốc, trimap, background) tương tự như phần trình bày phía trên của notebook. Sau này dự kiến sẽ chuyển qua dùng tham số dòng lệnh.

**Thực thi (thời gian xử lý với 1 ảnh):**

In [5]:
%%time
!python3 sequential.py

====SUCCESS GLOBAL MATTING====
====SUCCESS LOCAL MATTING====
CPU times: user 190 ms, sys: 26.7 ms, total: 217 ms
Wall time: 30.9 s


**Ảnh kết quả**

- Ảnh chỉ áp dụng Global Matting:

In [6]:
%%html
<img width="40%" src='https://drive.google.com/uc?id=1-a026cY2ST8ct9Sd-ZKhw2PBHWF4_qfY'/>

- Ảnh áp dụng cả Global và Local Matting:

In [7]:
%%html
<img width="40%" src='https://drive.google.com/uc?id=1-MXY0dxhLKQyfUArGLJMkTjWfluOW-MT'/>

#### **3.2.3. Thực thi với Python (có jit):**

Thực thi tuần tự với Numba jit. Phần sử dụng git:
- Hàm `global_alpha_matting`: hàm chạy phương pháp Gauss Siedel để tìm ra alpha phù hợp.

Những phần sử dụng thư viện có sẵn:
- Numpy: các phương thức `gradient`, `maximum`, `minimun`...; các phép toán nhân, cộng ma trận
- OpenCV: đọc ảnh, lưu ảnh, chuyển ảnh sang grayscale, RGB

> Nhóm đã có thử cài đặt để dùng jit với các hàm trên nhưng thấy tốc độ giảm không đáng kể (< 0.5s hoặc đôi khi chậm hơn) nên quyết định không jit nữa.

**Thực thi:**

In [8]:
%%time
!python3 sequential_jit.py

====SUCCESS GLOBAL MATTING====
====SUCCESS LOCAL MATTING====
CPU times: user 38.5 ms, sys: 14.8 ms, total: 53.2 ms
Wall time: 4.14 s


**Ảnh kết quả**

- Ảnh chỉ áp dụng Global Matting:


In [9]:
%%html
<img width="40%" src='https://drive.google.com/uc?id=1-DiphlaTa2wYUlTZiHGylhtlIxzPoUgr'/>

- Ảnh áp dụng cả Global và Local Matting:


In [10]:
%%html
<img width="40%" src='https://drive.google.com/uc?id=1-BxQMpPPH2iIdH_7j25BcWU_qf4wltiE'/>

#### **3.2.4. So sánh độ chính xác:**

In [11]:
img_seq = cv2.imread(os.path.join(IMAGES_FOLDER_PATH, '../Output/output_4_global.png'))
img_seq = cv2.cvtColor(img_seq, cv2.COLOR_BGR2RGB)
img_seq_jit = cv2.imread(os.path.join(IMAGES_FOLDER_PATH, '../Output/output_4_global_jit.png'))
img_seq_jit = cv2.cvtColor(img_seq_jit, cv2.COLOR_BGR2RGB)
errorL2 = cv2.norm(img_seq, img_seq_jit, cv2.NORM_L2)
similarity = 1 - errorL2 / ( img_seq.shape[0] * img_seq.shape[1] )
print('Độ giống nhau giữa ảnh của phiên bản tuần tự không jit và có jit = ',similarity)

Độ giống nhau giữa ảnh của phiên bản tuần tự không jit và có jit =  1.0


## **4. Cài đặt song song:**

### **4.1. Phân tích:**

**Phần thực hiện song song:** Bước tính toán alpha nằm trong hàm `global_alpha_matting`.

Thông tin về hàm `global_alpha_matting`:
  
    """
    Hàm thực thi global alpha matting

    Parameters:
    -----------
    alpha: giá trị xấp xỉ của alpha
    d2alpha: tổng đạo hàm riêng bậc 2 của alpha theo X và Y
    unknown_seg: mảng munby đại diện cho vùng chưa xác định rõ
    iters: số lần chạy của phương pháp ước lượng Gauss Siedel
    threshold: ngưỡng cho sự thay đổi sau mỗi lần chạy. Nếu dưới ngưỡng thì phương trình sẽ dừng lại
    beta: nhân tử beta của phương pháp ước lượng Gauss Siedel

    Return:
    -----------
    Mảng numpy đại diện cho chỉ số matte đã được tạo
    """

Quy trình của hàm:

- Bước đầu gán prev_alpha: 1 ma trận toàn số 0
- Tính toán sai khác giữa prev_alpha và alpha
- Đi vào vòng lặp với số lần lặp là iters
  - Tính độ khác biệt giữa prev_alpha và alpha. Nếu nhỏ hơn threshold thì kết thúc
  - Chạy 2 vòng lặp for lần lượt duyệt qua từng pixel của alpha
  - Tính lại giá trị của từng pixel theo công thức từ beta, alpha, pre_alpha, d2alpha (*)

=> Nhóm lựa chọn song song phần (*) ở trên bằng cách: chia cho mỗi thread đảm nhận một pixel để tính lại giá trị của alpha.

Nhóm chỉ song song khi xử lý global matting, còn local matting thì chỉ thực hiện trên jit.

**Lý do nhóm lựa chọn phần này:**

- Nhóm nhận thấy bước này có những công việc lặp đi lặp lại rất nhiều lần (tối đa là `iter`). Ở mỗi lần công việc có thể chia cho các thread thực hiện để giúp tăng tốc độ.
- Nhóm đã thực hiện song song hoá các phần khác như hàm tính gradient, diff,... của numpy hoặc chuyển ảnh rgb -> grayscale của opencv tuy nhiên kết quả không thay đổi nhiều (thậm chí chậm hơn từ 0.5s - 1s) nên không đưa vào bài làm.

### **4.2. Thiết kế:**

Tại mỗi thread:
- Xác định vị trí của nó
- Kiểm tra xem vị trí của nó có nằm trong bức ảnh hay không
- Nếu vị trí của thread là chưa xác định được foreground hay background thì ta sẽ tính giá trị alpha ở đây bằng công thức Gauss Siedel Iteration.


$$ {f_{i,j}}^{(n+1)} = \frac{\beta}{4}({f_{i+1,j}}^{(n)} + {f_{i-1,j}}^{(n+1)} + {f_{i,j+1}}^{(n)} + {f_{i,j-1}}^{(n+1)} - S_{i,j}) + (1-\beta){f_{i,j}}^{(n)}$$

Với: $$ \frac{\partial^2 f}{\partial x^2}+\frac{\partial^2 f}{\partial y^2} = S$$

Ví dụ ở pixel [1,1], alpha được tính như sau:

`alpha[1,1] = beta/4*(alpha[0,1] + alpha[1,0] + prev_alpha[2,1] + prev_alpha[1,2] - S[1,1]) + (1-beta)*prev_alpha[1,1]`

### **4.3. Đánh giá:**

#### **4.3.1. Kịch bản thí nghiệm:**

Chương trình sẽ lần lượt thực hiện theo quy trình sau:
1. Đọc ảnh gốc, ảnh tri map, ảnh nền mới. Chuyển ảnh gốc sang dạng grayscale
2. Chạy hàm `matting_combined`: hàm tổng hợp quy trình xử lý ảnh với Global Matting bên trên. 
  - Ở hàm tổng hợp sẽ gọi hàm `global_alpha_matting`
  - Bên trong hàm `global_alpha_matting` sẽ gọi hàm `compute_alpha_kernel` truyền vào thêm `block_size` và `grid_size` để chạy song song trên GPU.
3. Lưu file ảnh output được áp dụng Global Matting.
4. Bắt đầu pha Local Matting. Cho người dùng nhập (ở đây nhóm set cứng) giá trị của vùng mong muốn được cải thiện.
5. Tính toán giá trị `alpha` mới của các vùng này bằng các hàm tuần tự chứ không chạy song song như ở Global Matting.
6. Gọi hàm `alpha_blend` để áp giá trị `alpha` mới vào ảnh.
7. Lưu file ảnh output được áp dụng cả Global và Local Matting.

Nhìn chung, kịch bản thí nghiệm tương tự với thực thi tuần tự, chỉ khác ở bước 2 và một chút lưu ý ở bước 5. Global Matting thì sẽ có song song trên GPU, còn Local Matting thì chỉ chạy với jit.


#### **4.3.2. Thực thi:**

**Thực thi:**

In [12]:
%%time
!nvprof --print-gpu-trace python3 parallel.py

==315== NVPROF is profiling process 315, command: python3 parallel.py
====SUCCESS GLOBAL MATTING====
====SUCCESS LOCAL MATTING====
==315== Profiling application: python3 parallel.py
==315== Profiling result:
   Start  Duration            Grid Size      Block Size     Regs*    SSMem*    DSMem*      Size  Throughput  SrcMemType  DstMemType           Device   Context    Stream  Name
369.22ms  337.40us                    -               -         -         -         -  2.3438MB  6.7837GB/s    Pageable      Device     Tesla T4 (0)         1         7  [CUDA memcpy HtoD]
370.33ms  364.22us                    -               -         -         -         -  2.3438MB  6.2843GB/s    Pageable      Device     Tesla T4 (0)         1         7  [CUDA memcpy HtoD]
371.56ms  378.20us                    -               -         -         -         -  2.3438MB  6.0519GB/s    Pageable      Device     Tesla T4 (0)         1         7  [CUDA memcpy HtoD]
372.44ms  28.223us                    -           

**Ảnh kết quả**

- Ảnh chỉ áp dụng Global Matting:


In [13]:
%%html
<img width="40%" src='https://drive.google.com/uc?id=1-a3THTLnRgomDjLfPM3l4qcGGGnaJPdP'/>

- Ảnh áp dụng cả Global và Local Matting:


In [14]:
%%html
<img width="40%" src='https://drive.google.com/uc?id=1ZvkB_la4tmPchvHqGi6s0ngeAwwnTrng'/>

#### **4.3.3. So sánh độ chính xác:**

In [15]:
img_seq = cv2.imread(os.path.join(IMAGES_FOLDER_PATH, '../Output/output_4_global_jit.png'))
img_seq = cv2.cvtColor(img_seq, cv2.COLOR_BGR2RGB)
img_par = cv2.imread(os.path.join(IMAGES_FOLDER_PATH, '../Output/output_4_global_parallel.png'))
img_par = cv2.cvtColor(img_par, cv2.COLOR_BGR2RGB)
errorL2 = cv2.norm(img_seq, img_par, cv2.NORM_L2)
similarity = 1 - errorL2 / ( img_seq.shape[0] * img_seq.shape[1] )
print('Độ giống nhau giữa ảnh của phiên bản tuần tự và phiên bản song song 1 = ',similarity)

Độ giống nhau giữa ảnh của phiên bản tuần tự và phiên bản song song 1 =  0.9791598824421006


#### **4.3.4. Kết quả và giải thích:**

**Kết quả:**

Các kết quả thử nghiệm của nhóm đến thời điểm hiện tại:


- Về mặt độ chính xác: Ảnh của phiên bản song song chỉ khớp khoảng 98% so với ảnh từ phiên bản tuần tự.

- Về mặt thời gian: 5.14s

**Giải thích kết quả:**
- Kết quả ảnh khác nhau do thứ tự thực hiện tính toán mỗi pixel của 2 phiên bản khác nhau:
  + Tuần tự: từ trái sang phải, từ trên xuống dưới
  + Song song: ngẫu nhiên trong block
- Thời gian của phiên bản song song và phiên bản tuần tự có dùng jit không chênh lệch nhiều: có thể do số phần song song hoá của nhóm chưa nhiều, nên tốc độ không cải thiện nhiều.

## **5. Cài đặt song song - Phiên bản tối ưu hoá 1:**

### **5.1. Phân tích:**

**Phần thực hiện tối ưu:** Nhóm thực hiện cải thiện phần được song song hoá ở trên bằng cách sử dụng SMEM.

**Lý do nhóm lựa chọn phần này:** Nhóm nhận thấy là việc tính toán giá trị của mỗi pixel có phụ thuộc vào các pixel khác lân cận. Do đó sử dụng SMEM để lưu trữ mảng giá trị mỗi block để giảm thiểu thời gian truy xuất.

### **5.2. Thiết kế:**

Tại mỗi thread:
- Xác định vị trí của nó trong tổng thể grid và bên trong block
- Kiểm tra xem vị trí của nó có nằm trong bức ảnh hay không
- Sao chép dữ liệu tương ứng của các mảng `alpha`, `prev_alpha`, `d2_alpha` qua SMEM
- Nếu vị trí của thread là chưa xác định được foreground hay background thì ta sẽ tính giá trị alpha ở đây bằng công thức Gauss Siedel Iteration.


$$ {f_{i,j}}^{(n+1)} = \frac{\beta}{4}({f_{i+1,j}}^{(n)} + {f_{i-1,j}}^{(n+1)} + {f_{i,j+1}}^{(n)} + {f_{i,j-1}}^{(n+1)} - S_{i,j}) + (1-\beta){f_{i,j}}^{(n)}$$

Với: $$ \frac{\partial^2 f}{\partial x^2}+\frac{\partial^2 f}{\partial y^2} = S$$

Trong công thức thì ta sử dụng các index là vị trí của thread trong block.

**Vấn đề gặp phải:**

Tại công thức tính thì giá trị alpha có phụ thuộc vào những pixel liền kề. Do đó ở các vị trí biên của block thì ta sẽ lấy giá trị `alpha` và `prev_alpha` từ mảng lớn được chép từ host sang.

Ví dụ ở pixel [0,1], alpha sẽ được tính như sau, [0,1] ở đây là toạ độ trong block của pixel:

`alpha[0,1] = beta/4*(s_alpha[-1,0] + s_alpha[0,0] + s_prev_alpha[1,1] + s_prev_alpha[0,2] - S[0,1]) + (1-beta)*s_prev_alpha[0,1]`

Ta nhận thấy `s_alpha[-1,0]` nằm ngoài vùng shared mem block của mình. Do đó ta thay thế `s_alpha[-1,0]` bằng `alpha` tương ứng (mảng đã được copy sang device trước đó).

### **5.3. Đánh giá:**

#### **5.3.1. Kịch bản thí nghiệm:**

Chương trình sẽ lần lượt thực hiện theo quy trình sau:
1. Đọc ảnh gốc, ảnh tri map, ảnh nền mới. Chuyển ảnh gốc sang dạng grayscale
2. Chạy hàm `matting_combined`: hàm tổng hợp quy trình xử lý ảnh với Global Matting bên trên. 
- Ở hàm tổng hợp sẽ gọi hàm `global_alpha_matting`
- Ở đây sẽ gọi hàm `compute_alpha_kernel` truyền vào thêm `block_size` và `grid_size`, hàm sẽ chạy song song trên GPU.
3. Lưu file ảnh output được áp dụng Global Matting.
4. Bắt đầu pha Local Matting. Cho người dùng nhập (ở đây nhóm set cứng) giá trị của vùng mong muốn được cải thiện.
5. Tính toán giá trị `alpha` mới của các vùng này bằng các hàm tuần tự chứ không chạy song song như ở Global Matting.
6. Gọi hàm `alpha_blend` để áp giá trị `alpha` mới vào ảnh.
7. Lưu file ảnh output được áp dụng cả Global và Local Matting.

Nhìn chung, kịch bản thí nghiệm tương tự với phiên bản song song. Tuy nhiên tại bước 2 thì hàm `compute_alpha_kernel` tính toán trên SMEM.


#### **5.3.2. Thực thi:**

**Thực thi:**

In [16]:
%%time
!nvprof --print-gpu-trace python3 parallel_opt_v1.py

==335== NVPROF is profiling process 335, command: python3 parallel_opt_v1.py
====SUCCESS GLOBAL MATTING====
====SUCCESS LOCAL MATTING====
==335== Profiling application: python3 parallel_opt_v1.py
==335== Profiling result:
   Start  Duration            Grid Size      Block Size     Regs*    SSMem*    DSMem*      Size  Throughput  SrcMemType  DstMemType           Device   Context    Stream  Name
353.60ms  338.42us                    -               -         -         -         -  2.3438MB  6.7632GB/s    Pageable      Device     Tesla T4 (0)         1         7  [CUDA memcpy HtoD]
354.72ms  335.86us                    -               -         -         -         -  2.3438MB  6.8147GB/s    Pageable      Device     Tesla T4 (0)         1         7  [CUDA memcpy HtoD]
355.98ms  381.11us                    -               -         -         -         -  2.3438MB  6.0057GB/s    Pageable      Device     Tesla T4 (0)         1         7  [CUDA memcpy HtoD]
357.03ms  27.968us                  

**Ảnh kết quả**

- Ảnh chỉ áp dụng Global Matting:


In [17]:
%%html
<img width="40%" src='https://drive.google.com/uc?id=1WeUnf6TYkWmTrSBSbdfn1FKv3PigCtcj'/>

- Ảnh áp dụng cả Global và Local Matting:


In [18]:
%%html
<img width="40%" src='https://drive.google.com/uc?id=1OGbbkjKtehe72X6WTObowolKBygDor4f'/>

#### **5.3.3. So sánh độ chính xác:**

In [19]:
img_seq = cv2.imread(os.path.join(IMAGES_FOLDER_PATH, '../Output/output_4_global_jit.png'))
img_seq = cv2.cvtColor(img_seq, cv2.COLOR_BGR2RGB)
img_par = cv2.imread(os.path.join(IMAGES_FOLDER_PATH, '../Output/output_4_global_parallel.png'))
img_par = cv2.cvtColor(img_par, cv2.COLOR_BGR2RGB)
img_par_v1 = cv2.imread(os.path.join(IMAGES_FOLDER_PATH, '../Output/output_4_global_parallel_v1.png'))
img_par_v1 = cv2.cvtColor(img_par_v1, cv2.COLOR_BGR2RGB)
errorL2 = cv2.norm(img_seq, img_par_v1, cv2.NORM_L2)
similarity = 1 - errorL2 / ( img_seq.shape[0] * img_seq.shape[1] )
print('Độ giống nhau giữa ảnh của phiên bản tuần tự và phiên bản song song 1 = ',similarity)
errorL2_2 = cv2.norm(img_par, img_par_v1, cv2.NORM_L2)
similarity_2 = 1 - errorL2_2 / ( img_par.shape[0] * img_par.shape[1] )
print('Độ giống nhau giữa ảnh của phiên bản tuần tự và phiên bản song song 1 = ',similarity_2)

Độ giống nhau giữa ảnh của phiên bản tuần tự và phiên bản song song 1 =  0.9791908386586683
Độ giống nhau giữa ảnh của phiên bản tuần tự và phiên bản song song 1 =  0.9994664039305301


#### **5.3.4. Kết quả và giải thích:**

**Kết quả:**

Các kết quả thử nghiệm của nhóm đến thời điểm hiện tại:


- Về mặt độ chính xác: Ảnh output phiên bản cải tiến gần như giống với phiên bản song song, do đó nó cũng có độ khớp với output phiên bản có jit là 98%.

- Về mặt thời gian: 5.04s


**Giải thích kết quả:**
- Kết quả ảnh khác nhau do thứ tự thực hiện tính toán mỗi pixel của 2 phiên bản khác nhau:
  + Tuần tự: từ trái sang phải, từ trên xuống dưới
  + Song song: ngẫu nhiên trong block và chưa xử lý được các phần tử ở biên của mỗi block
- Thời gian của phiên bản song song và phiên bản tuần tự có dùng jit không chênh lệch nhiều: có thể do số phần song song hoá của nhóm chưa nhiều, nên tốc độ không cải thiện nhiều.

## **6. Tổng hợp kết quả và giải thích**

### **6.1. Các kết quả:**

**Về mặt độ chính xác:** Độ khớp của ảnh từ các phiên bản:

|       **Phiên bản**      	| **Tuần tự có jit** 	| **Song song** 	| **Song song + Cải tiến** 	|
|:------------------------:	|:------------------:	|:-------------:	|:------------------------:	|
|    **Tuần tự có jit**    	|          -         	|      98%      	|            98%           	|
|       **Song song**      	|          -         	|       -       	|           99.9%           	|
| **Song song + Cải tiến** 	|          -         	|       -       	|             -            	|



**Về mặt thời gian:** Ở đây khi gom tất cả các phiên bản vào chung 1 notebook và chạy theo thứ tự: Tuần tự không jit -> Tuần tự có jit -> Song song -> Cải tiến thì thời gian chạy khác với khi chạy riêng lẻ từng phiên bản. Số liệu có thể khác qua mỗi lần chạy. Thời gian chạy hiển thị ở các cell phía trên là khi chạy chung trên 1 notebook.

|       **Phiên bản**      	| **Chạy chung trên 1 notebook** 	| **Chạy riêng lẻ** 	|
|:------------------------:	|:------------------------------:	|:-----------------:	|
|   **Tuần tự không jit**  	|              24.4s             	|       27.8s       	|
|    **Tuần tự có jit**    	|              4.14s             	|       9.35s       	|
|       **Song song**      	|              5.14s             	|       6.04s       	|
| **Song song + Cải tiến** 	|              5.04s             	|       5.63s       	|

### **6.2. Giải thích các kết quả:**

**Về mặt độ chính xác**
- Các phiên bản song song có sự chính xác khác so với bản tuần tự là do thứ tự thực hiện tính toán mỗi pixel của 2 phiên bản khác nhau:
  + Tuần tự: từ trái sang phải, từ trên xuống dưới
  + Song song: ngẫu nhiên trong block
- Hai phiên bản song song khác nhau một chút có thể do cách làm tròn số của hệ thống.

**Về mặt thời gian**
- Khi chạy riêng lẻ và gộp chung có thời gian khác nhau là bởi vì do trong các file .py nhiều hàm trùng lặp. Nhưng phiên bản sau hệ thống sẽ tốn ít thời gian để biên dịch hơn, vì đã từng được biên dịch một lần trước đó => Để so sánh thì dùng kết quả khi chạy riêng lẻ là hợp lý.
- Các mốc thời gian đúng như ta kì vọng:
(Song song cải tiến) < (Song song) < (Tuần tự có jit) < (Tuần tự không jit)

## **7. Nhìn lại quá trình làm đồ án**

**1712898 - Nguyễn Văn Hậu**
- Khó khăn
    + Xuất phát điểm là người may mắn khi tham gia vào lớp Lập trình song song ứng dụng nhưng chưa học qua lớp tiên quyết trước đó. Khó khăn ban đầu em gặp phải là quy trình giải quyết một bài toán song song, kĩ thuật để tối ưu hóa các thuật toán bằng xử lý song song. Thời gian đầu khá khó nhằn để tìm được đề tài vừa với sức mình. May mắn hơn khi em có một team nhiều kinh nghiệm, từng bước giúp em theo được quá trình giải quyết bài toán.
- Học được:
    + Cách phân chia công việc tốt, giúp cho quá trình làm việc diễn ra khá suôn sẻ. Cách hoạt động trong nhóm, tính cách của những con người trong nhóm, giúp em biết em thích hợp với nhóm như thế nào, cũng là bài học chọn người đồng hành cho cơ hội làm việc sau này.
    + Em biết và hiểu thêm về luồng xử lý song song trên GPU, cách ứng dụng vào một bài toán thực tế.

**18120181 - Nguyễn Thị Cẩm Hồng**
- Khó khăn
    + Thời gian còn hạn chế do có khá nhiều môn học và môn nào cũng có nhiều đồ án, deadline. Nhất là giai đoạn cuối kì khi có nhiều đồ án môn học và em có đi thực tập.
    + Tìm hiểu về bản chất của công thức Poisson, cách tự tạo trimap.
    + Giải thích các kết quả gặp phải trong bài toán.
- Học được:
    + Cách lập trình song song bằng numba cuda.
    + Cải thiện nhiều khả năng làm việc nhóm, quản lý version github.

**18120216 - Mai Huỳnh Trung Nguyên**
- Khó khăn
    + Học kì này, em có nhiều môn học khá nặng về cả lý thuyết và thực hành. 
    + Nhiều lúc hơi nản chí vì không có hướng giải quyết cho vấn đề của bài toán
- Học được:
    + Sự chỉn chu, tỉ mỉ, rõ ràng hơn khi viết báo cáo môn học
    + Lắng nghe bài làm của nhóm và cách giải thích phần của mình cho dễ hiểu
    + Song song hoá bằng Python

**Nhóm**: 
- Khó khăn:
  + Lúc đầu nhóm bị mông lung về các đề tài lựa chọn, nhóm tập trung vào các dự án xử lý ảnh thuần không xử lý các mạng neuron phức tạp => khá ít bài toán
  + Khi nhóm có dự án mẫu rồi, làm được một thời gian thì phát hiện dự án mẫu làm nhiều cái sai sót, nhiều phần dư thừa => phải fix bug, cải thiện nhiều. Bên cạnh đó còn thấy được nhiều dự án khác có độ hiệu quả cao hơn.
  + Về nửa sau của dự án, nhóm thấy không có quá nhiều phần để song song hoá, khi cài đặt xong thì không thấy cải thiện nhiều về hiệu suất; nhóm mất nhiều thời gian để nghiên cứu các phương pháp mới, thử nhiều cách nhưng không thành công.
- Do đó, nếu có thêm thời gian nhóm sẽ cố gắng thực hiện:
  + Tìm hiểu thêm về cách song song hoá để tối ưu thời gian rõ rệt
  + Thử nghiệm trên nhiều ví dụ hơn để hiểu hơn nữa về dự án


## **8. Tài liệu tham khảo**

- Project: Poisson Matting, [github](https://github.com/avani17101/Poisson-Matting).
- Paper: Poisson Matting của Juan Sun, Jiaya Jia, Chi-Keung Tang, Heung-Yeung Shum, [paper](http://www.cse.cuhk.edu.hk/~leojia/all_final_papers/matting_siggraph04.pdf)